In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from collections import OrderedDict
import warnings; warnings.filterwarnings("ignore")

In [4]:
X_train = pd.read_csv("../data/train/feature.csv")
y_train = pd.read_csv("../data/train/label.csv")
X_test  = pd.read_csv("../data/test/feature.csv")
y_test  = pd.read_csv("../data/test/label.csv")
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(73500, 1001)
(73500, 3)
(24500, 1001)
(24500, 3)


In [5]:
v_names = X_train.columns[1:]
df_train = pd.merge(X_train, y_train, on= 'APPLICATION_ID')
df_train.head()

,APPLICATION_ID,v000,v001,v002,v003,v004,v005,v006,v007,v008,...,v992,v993,v994,v995,v996,v997,v998,v999,APPLICATION_DATE,DEFAULT_LABEL
0,44c4e0d49d5bf94efa446e03ecf84ba5,NaN,NaN,NaN,5.528423,NaN,NaN,10.0,0.0,0.016305,...,NaN,NaN,NaN,NaN,41.0,0.0,NaN,0.0,2021-09-14,0
1,993ee0eed4ad96f4f5d6dc30d56bdf94,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,NaN,...,NaN,NaN,NaN,NaN,8.0,0.0,NaN,0.0,2021-07-06,0
2,a9815449c0a385ee0d53305b8cca95c6,NaN,NaN,NaN,5.062508,0.00000,NaN,7.0,0.0,1.039721,...,NaN,NaN,NaN,NaN,22.0,0.0,NaN,0.0,2021-06-10,0
3,4f9dd0d609cee4360e110d56cc35afd9,0.0,NaN,NaN,2.575101,0.44461,NaN,NaN,NaN,0.980110,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,2021-07-04,0
4,904de5e19a5ea36e8f4b9f4cf5bdd953,0.0,NaN,0.000473,5.528423,0.00000,NaN,1.0,0.0,0.950612,...,NaN,0.079084,0.415888,NaN,76.0,0.0,3.894232,0.0,2021-06-23,0


In [9]:
# 如果是连续性数据，首先排序分为5组，离散化之后，计算WOE
# 如果是离散型数据，直接分类就好（0-10等）, 计算WOE
label = "DEFAULT_LABEL"
n_bins =  5
B_total = df_train[label].sum()           # default, y = 1
G_total = len(df_train[label]) - B_total  # not default, y = 0
diverse_threshold = 30
WOE_list = []
woe_is = []
iv_is = []
for v_name in v_names:
    data_type = 'discrete'
    df_v = df_train[[v_name, label]]
    df_v.dropna(inplace = True)

    if len(df_v[v_name].unique()) > diverse_threshold:
        # continuous 离散化
        # https://scikit-learn.org/stable/modules/preprocessing.html#discretization
        data_type = 'continous'
        data = preprocessing.KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile').fit_transform(df_v[[v_name]])
        df_v[v_name] = data  # 离散化
    df_data = pd.pivot_table(df_v, index = v_name,columns=label, values=label, aggfunc=len, fill_value=0)
    print(df_data[1])
    df_data[0] = df_data[0] / G_total   # good y=0
    df_data[1] = df_data[1] / B_total   # bad  y=1
    ratio = np.log(df_data[0] / df_data[1])
    ratio.replace([np.inf, -np.inf], 0, inplace=True)
    print(np.array(df_data[1]))
    A_is = df_data
    break
    # save
#     print(ratio)
#     print(np.array(ratio))
#     break

    woe_i = np.array(ratio)
    woe_is.append(woe_i)
    iv_i = []
    print((df_data[0] - df_data[1]) * woe_i[i])
    break
    for i in range(len(woe_i)):
        iv_i.append((df_data[0] - df_data[1]) * woe_i[i])
    iv_i.append(sum(iv_i))
    print(iv_i)
    break
    iv_is.append(iv_i)
    tmp_dict = OrderedDict()
    tmp_dict["name"] = v_name
    tmp_dict["WOE"] = ratio.sum()
    tmp_dict['Type'] = data_type
    WOE_list.append(tmp_dict)

WOE_result = pd.DataFrame(WOE_list)
WOE_result.head()


v000
0.0    670
1.0      3
2.0      0
3.0      0
Name: 1, dtype: int64
[0.43421905 0.00194426 0.         0.        ]


""


In [10]:
woe_is

[array([0.06977241, 0.27940339, 0.        , 0.        ]),
 array([0.0642837]),
 array([-0.28959287, -0.29220043,  0.32690573, -0.38182619,  0.32947973]),
 array([0.10751372, 0.10747806, 0.11500866, 0.02101769, 0.20939511]),
 array([0.01050698, 0.07816849, 0.11847302, 0.00486246]),
 array([0.02060825, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 array([0.06769036, 0.09309664, 0.14426011, 0.19854584]),
 array([0.1206539]),
 array([ 0.09850168, -0.00353691, -0.0162923 ,  0.23261475,  0.123709  ]),
 array([-0.1839199 ,  0.04372382, -0.03182037,  0.01295216,  0.21778353]),
 array([-0.013892  ,  0.07832191,  0.07926642,  0.07422537]),
 array([ 0.07312206,  0.18003092, -0.70684593,  0.        , -1.3574335 ,
         0.        ,  0.        ,  0.        ,  0.        ]),
 array([0.03753526, 0.00871243

In [11]:
iv_is

[[v000
  0.0    0.002189
  1.0    0.000044
  2.0    0.000015
  3.0    0.000006
  dtype: float64,
  v000
  0.0    0.008767
  1.0    0.000175
  2.0    0.000058
  3.0    0.000023
  dtype: float64,
  v000
  0.0    0.0
  1.0    0.0
  2.0    0.0
  3.0    0.0
  dtype: float64,
  v000
  0.0    0.0
  1.0    0.0
  2.0    0.0
  3.0    0.0
  dtype: float64,
  v000
  0.0    0.010957
  1.0    0.000219
  2.0    0.000073
  3.0    0.000029
  dtype: float64],
 [v001
  0.0    0.000432
  dtype: float64,
  v001
  0.0    0.000432
  dtype: float64],
 [v002
  0.0    0.000519
  1.0    0.000523
  2.0   -0.000435
  3.0    0.000715
  4.0   -0.000439
  dtype: float64,
  v002
  0.0    0.000524
  1.0    0.000528
  2.0   -0.000439
  3.0    0.000721
  4.0   -0.000443
  dtype: float64,
  v002
  0.0   -0.000586
  1.0   -0.000591
  2.0    0.000492
  3.0   -0.000807
  4.0    0.000496
  dtype: float64,
  v002
  0.0    0.000684
  1.0    0.000690
  2.0   -0.000574
  3.0    0.000942
  4.0   -0.000579
  dtype: float64,
  v002


In [99]:
WOE_result

,name,WOE,Type
0,v000,0.349176,discrete
1,v001,-14.226897,discrete
2,v002,-0.307234,continous
3,v003,0.560413,continous
4,v004,0.212011,continous
...,...,...,...
995,v995,0.200796,continous
996,v996,0.599501,continous
997,v997,0.810020,discrete
998,v998,0.126141,continous
